In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn import svm

import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from jupyterthemes import jtplot


jtplot.style('monokai') # dark background for matplotlib

# Load and Scale Data

## Load

In [ ]:
pddata_train = pd.read_csv('Data/sign_mnist_train.csv')
data_train = pddata_train.values
data_train_X = data_train[:,1:]
data_train_y = data_train[:,:1].reshape(-1) # turn to 1d


pddata_test = pd.read_csv('Data/sign_mnist_test.csv')
data_test = pddata_test.values
data_test_X = data_test[:, 1:]
data_test_y = data_test[:, :1].reshape(-1) # turn to 1d

## Scale

In [ ]:
mmscaler = MinMaxScaler()
mmscaled_train_X = mmscaler.fit_transform(data_train_X)
mmscaled_test_X = mmscaler.transform(data_test_X)

stdscaler = StandardScaler()
stdscaled_train_X = stdscaler.fit_transform(data_train_X)
stdscaled_test_X = stdscaler.transform(data_test_X)

# Logistic Regression

In [ ]:
def logreg(penalty='l2', solver='lbfgs', r=None, use_minmax_dataset=True, verbose=False) -> None:
    _iter = 100000
    # use minmax or standard scaling
    _train_X = mmscaled_train_X if use_minmax_dataset else stdscaled_train_X
    _test_X = mmscaled_test_X if use_minmax_dataset else stdscaled_test_X

    # place to store results
    train_accs = []
    test_accs = []
    cs = []

    for exponent in (range(-3,3+1) if r is None else r):
        if verbose:
            print(f'loop: {exponent}')
            
        c_value = 10**exponent
        cs.append(c_value) # for graphing

        logreg = LogisticRegression(max_iter=_iter, C=c_value, penalty=penalty, solver=solver, verbose=verbose)
        logreg.fit(_train_X, data_train_y.reshape(-1)) # fit model 

        # if verbose:
        #     print('fitting done')

        # store results
        train_accs.append(np.mean(np.equal(logreg.predict(_train_X), data_train_y))) # train accuuracy
        test_accs.append(np.mean(np.equal(logreg.predict(_test_X), data_test_y)))  # test accuracy

        # if verbose:
        #     print(f"\ttrain: {train_accs[-1]}")
        #     print(f"\ttest: {test_accs[-1]}")

    # no reg
    cs.append(10000000)
    logreg = LogisticRegression(max_iter=_iter, C=10000000, penalty=penalty, solver=solver, verbose=verbose)
    logreg.fit(_train_X, data_train_y.reshape(-1)) # fit model 
    train_accs.append(np.mean(np.equal(logreg.predict(_train_X), data_train_y))) # train accuuracy
    test_accs.append(np.mean(np.equal(logreg.predict(_test_X), data_test_y)))  # test accuracy


    pd.DataFrame([cs, train_accs, test_accs]).transpose().to_csv(f'Figures/logreg_{penalty}_{"minmax" if use_minmax_dataset else "std"}_results.csv', header=False, index=False)

    
    plt.title(f'Acc v. C value; Log. Reg. {penalty} regularization, {"minmax" if use_minmax_dataset else "std"} scaling')
    plt.xlabel("C values")
    plt.ylabel("Accuracy")
    # plt.xscale('log') # don't need log scaling if not displaying x-axis
    plt.plot(cs, train_accs, label='train')
    plt.plot(cs, test_accs, label='test')
    plt.legend(loc='lower right')
    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=False
        )
    plt.savefig(f'Figures/logreg_{penalty}_{"minmax" if use_minmax_dataset else "std"}_results.png', dpi=500)
    plt.show()

    return train_accs, test_accs, cs

In [ ]:
logreg_l2_minmax_results = logreg(penalty='l2', use_minmax_dataset=True, verbose=True)

In [ ]:
logreg_l2_std_results = logreg(penalty='l2', use_minmax_dataset=False, verbose=True)

In [ ]:
logreg_l1_minmax_results = logreg(penalty='l1', solver='saga', use_minmax_dataset=True, verbose=True)

In [ ]:
logreg_l1_std_results = logreg(penalty='l1', solver='saga', use_minmax_dataset=False, verbose=True)

# Support Vector Machine

In [ ]:
def SVM(kernel='linear', r=None, use_minmax_dataset=True, degree=2, verbose: bool=False) -> None:
    _iter = 100000
    # use minmax or standard scaler
    _train_X = mmscaled_train_X if use_minmax_dataset else stdscaled_train_X
    _test_X = mmscaled_test_X if use_minmax_dataset else stdscaled_test_X

    # store results, to graph
    train_accs = []
    test_accs = []
    cs = []
    
    for exponent in (range(-3,3+1) if r is None else r):
        if verbose:
            print(f'loop: {exponent}')
            
        c = 10**exponent
        cs.append(c) # store, to graph

        S = svm.SVC(probability=False, max_iter=_iter, kernel=kernel, C=c, degree=degree, verbose=verbose)
        
        S.fit(_train_X, data_train_y)

        train_accs.append(np.mean(np.equal(S.predict(_train_X), data_train_y))) # train accuuracy
        test_accs.append(np.mean(np.equal(S.predict(_test_X), data_test_y)))  # test accuracy


    # no reg
    cs.append(10000000)
    S = svm.SVC(probability=False, max_iter=_iter, kernel=kernel, C=10000000, degree=degree, verbose=verbose)
    S.fit(_train_X, data_train_y.reshape(-1)) # fit model 
    train_accs.append(np.mean(np.equal(S.predict(_train_X), data_train_y))) # train accuuracy
    test_accs.append(np.mean(np.equal(S.predict(_test_X), data_test_y)))  # test accuracy


    pd.DataFrame([cs, train_accs, test_accs]).transpose().to_csv(f'Figures/svm_{f"polyD{degree}" if kernel=="poly" else kernel}_{"minmax" if use_minmax_dataset else "std"}_results.csv', header=False, index=False)

    plt.title(f'Acc v. C value; SVM {f"poly degree {degree}" if kernel=="poly" else kernel} kernel,  {"minmax" if use_minmax_dataset else "std"} scaling')
    plt.xlabel("C values")
    plt.ylabel("Accuracy")
    # plt.xscale('log') # don't need log scaling if not displaying x-axis
    plt.plot(cs, train_accs, label='train')
    plt.plot(cs, test_accs, label='test')
    plt.legend(loc='lower right')
    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=False
        )
    plt.savefig(f'Figures/svm_{f"polyD{degree}" if kernel=="poly" else kernel}_{"minmax" if use_minmax_dataset else "std"}_results.png', dpi=500)
    plt.show()

    return train_accs, test_accs, cs


In [ ]:
svm_linear_minmax_results = SVM(kernel='linear', use_minmax_dataset=True,verbose=True)

In [ ]:
svm_linear_std_results = SVM(kernel='linear', use_minmax_dataset=False,verbose=True)

In [ ]:
svm_rbf_minmax_results = SVM(kernel='rbf', use_minmax_dataset=True,verbose=True)

In [ ]:
svm_rbf_std_results = SVM(kernel='rbf', use_minmax_dataset=False,verbose=True)

In [ ]:
svm_polyd1_minmax_results = SVM(kernel='poly', degree=1, use_minmax_dataset=True, verbose=True)

In [ ]:
svm_polyd2_minmax_results = SVM(kernel='poly', degree=2, use_minmax_dataset=True, verbose=True)

In [ ]:
svm_polyd3_minmax_results = SVM(kernel='poly', degree=3, use_minmax_dataset=True, verbose=True)

In [ ]:
svm_polyd4_minmax_results = SVM(kernel='poly', degree=4, use_minmax_dataset=True, verbose=True)

In [ ]:
svm_polyd1_std_results = SVM(kernel='poly', degree=1, use_minmax_dataset=False,verbose=True)

In [ ]:
svm_polyd2_std_results = SVM(kernel='poly', degree=2, use_minmax_dataset=False,verbose=True)

In [ ]:
svm_polyd3_std_results = SVM(kernel='poly', degree=3, use_minmax_dataset=False,verbose=True)

In [ ]:
svm_polyd4_std_results = SVM(kernel='poly', degree=4, use_minmax_dataset=False,verbose=True)